# 数据预处理

In [146]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [147]:
# 读取CSV
raw_data = pd.read_csv("data/train.csv", header=0)

# 缺失值处理
raw_data = raw_data.fillna(0)

In [148]:
raw_data = raw_data.values
raw_data.shape

(10086, 19)

In [149]:
sline_num =10086
train_data = raw_data[0:sline_num, 0:18]
train_data.shape

(10086, 18)

In [150]:
train_label = raw_data[0:sline_num, 18:19]
t_arr = []
for line in train_label:
    t_arr.append(int(line[0]))
t_arr = np.array(t_arr)

In [151]:
test_data = raw_data[0:3000, 0:18]
test_label = raw_data[0:3000, 18:19]
test_label_arr = []
for line in test_label:
    test_label_arr.append(int(line[0]))
test_label_arr = np.array(test_label_arr)
test_data = torch.from_numpy(test_data).type(torch.FloatTensor)
print(test_data.shape)
test_label_arr = torch.from_numpy(test_label_arr).type(torch.LongTensor)
print(test_label_arr.shape)

torch.Size([3000, 18])
torch.Size([3000])


In [152]:
train_data = torch.from_numpy(train_data).type(torch.FloatTensor)
t_arr = torch.from_numpy(t_arr).type(torch.LongTensor)
print(train_data.shape)
print(t_arr.shape)

torch.Size([10086, 18])
torch.Size([10086])


# 模型

In [153]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(18,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,2)
    def forward(self,x):
        x = self.fc1(x)
        x = torch.tanh(x)
        x = self.fc2(x)
        x = torch.tanh(x)
        x = self.fc3(x)
        return x
             
    def predict(self,x):
        pred = F.softmax(self.forward(x))
        ans = []
        for t in pred:
            if t[0]>t[1]:
                ans.append(0)
            else:
                ans.append(1)
        return torch.tensor(ans)

In [161]:
def weighted_log_loss(p, y):
    sum_wi = 0
    sum_loss = 0
    for i in range(len(p)):
        wi = 1
        if y[i] == 1:
            wi = 2
        sum_wi += wi
        sum_loss += wi*(y[i]*math.log1p(p[i]) + (1-y[i])*math.log1p(1-p[i]))
    return -sum_loss/sum_wi

In [158]:
model = Net()
#criterion = nn.CrossEntropyLoss()
#criterion = sklearn.metrics.log_loss(x,y)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [159]:
epochs = 2000
losses = []
for i in range(epochs):
    y_pred = model.forward(train_data)
    loss = weighted_log_loss(y_pred,t_arr)
    losses.append(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.

In [160]:
from sklearn.metrics import accuracy_score
print(accuracy_score(model.predict(test_data),test_label_arr))

0.32266666666666666


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


In [ ]:
math.log(20)

In [ ]:
x = np.array([0.9,0.0, 1.1,0.2])
y = np.array([1,1, 1,0])
weighted_log_loss(x, y)